# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 15 2022 9:45AM,246291,21,627117,"NBTY Global, Inc.",Released
1,Sep 15 2022 9:43AM,246496,19,N17751,"NAPP Technologies, LLC",Released
2,Sep 15 2022 9:36AM,246494,10,SOTAH0000779,"Nextsource Biotechnology, LLC",Released
3,Sep 15 2022 9:32AM,246493,12,HH-36922,Hush Hush,Released
4,Sep 15 2022 9:32AM,246493,12,HH-36923,Hush Hush,Released
5,Sep 15 2022 9:32AM,246493,12,HH-36924,Hush Hush,Released
6,Sep 15 2022 9:32AM,246493,12,HH-36925,Hush Hush,Released
7,Sep 15 2022 9:32AM,246493,12,HH-36926,Hush Hush,Released
8,Sep 15 2022 9:32AM,246493,12,HH-36927,Hush Hush,Released
9,Sep 15 2022 9:32AM,246493,12,HH-36928,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
20,246490,Released,12
21,246491,Released,1
22,246493,Released,14
23,246494,Released,1
24,246496,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246490,NaN,NaN,12.0
246491,NaN,NaN,1.0
246493,NaN,NaN,14.0
246494,NaN,NaN,1.0
246496,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246291,0.0,0.0,1.0
246428,0.0,0.0,1.0
246429,0.0,0.0,1.0
246435,11.0,18.0,0.0
246462,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246291,0,0,1
246428,0,0,1
246429,0,0,1
246435,11,18,0
246462,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246291,0,0,1
1,246428,0,0,1
2,246429,0,0,1
3,246435,11,18,0
4,246462,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246291,,,1
1,246428,,,1
2,246429,,,1
3,246435,11,18,
4,246462,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 15 2022 9:45AM,246291,21,"NBTY Global, Inc."
1,Sep 15 2022 9:43AM,246496,19,"NAPP Technologies, LLC"
2,Sep 15 2022 9:36AM,246494,10,"Nextsource Biotechnology, LLC"
3,Sep 15 2022 9:32AM,246493,12,Hush Hush
17,Sep 15 2022 9:18AM,246491,19,ACG North America LLC
18,Sep 15 2022 9:01AM,246490,10,ISDIN Corporation
30,Sep 15 2022 8:59AM,246489,10,"ClearSpec, LLC"
31,Sep 15 2022 8:46AM,246487,10,TherapeuticsMD Sample
32,Sep 15 2022 8:33AM,246486,19,"GCH Granules USA, Inc."
33,Sep 15 2022 8:32AM,246483,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 15 2022 9:45AM,246291,21,"NBTY Global, Inc.",,,1
1,Sep 15 2022 9:43AM,246496,19,"NAPP Technologies, LLC",,,1
2,Sep 15 2022 9:36AM,246494,10,"Nextsource Biotechnology, LLC",,,1
3,Sep 15 2022 9:32AM,246493,12,Hush Hush,,,14
4,Sep 15 2022 9:18AM,246491,19,ACG North America LLC,,,1
5,Sep 15 2022 9:01AM,246490,10,ISDIN Corporation,,,12
6,Sep 15 2022 8:59AM,246489,10,"ClearSpec, LLC",,,1
7,Sep 15 2022 8:46AM,246487,10,TherapeuticsMD Sample,,,1
8,Sep 15 2022 8:33AM,246486,19,"GCH Granules USA, Inc.",,,1
9,Sep 15 2022 8:32AM,246483,10,ISDIN Corporation,,,44


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 15 2022 9:45AM,246291,21,"NBTY Global, Inc.",1,,
1,Sep 15 2022 9:43AM,246496,19,"NAPP Technologies, LLC",1,,
2,Sep 15 2022 9:36AM,246494,10,"Nextsource Biotechnology, LLC",1,,
3,Sep 15 2022 9:32AM,246493,12,Hush Hush,14,,
4,Sep 15 2022 9:18AM,246491,19,ACG North America LLC,1,,
5,Sep 15 2022 9:01AM,246490,10,ISDIN Corporation,12,,
6,Sep 15 2022 8:59AM,246489,10,"ClearSpec, LLC",1,,
7,Sep 15 2022 8:46AM,246487,10,TherapeuticsMD Sample,1,,
8,Sep 15 2022 8:33AM,246486,19,"GCH Granules USA, Inc.",1,,
9,Sep 15 2022 8:32AM,246483,10,ISDIN Corporation,44,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 15 2022 9:45AM,246291,21,"NBTY Global, Inc.",1,,
1,Sep 15 2022 9:43AM,246496,19,"NAPP Technologies, LLC",1,,
2,Sep 15 2022 9:36AM,246494,10,"Nextsource Biotechnology, LLC",1,,
3,Sep 15 2022 9:32AM,246493,12,Hush Hush,14,,
4,Sep 15 2022 9:18AM,246491,19,ACG North America LLC,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 15 2022 9:45AM,246291,21,"NBTY Global, Inc.",1.0,NaN,NaN
1,Sep 15 2022 9:43AM,246496,19,"NAPP Technologies, LLC",1.0,NaN,NaN
2,Sep 15 2022 9:36AM,246494,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
3,Sep 15 2022 9:32AM,246493,12,Hush Hush,14.0,NaN,NaN
4,Sep 15 2022 9:18AM,246491,19,ACG North America LLC,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 15 2022 9:45AM,246291,21,"NBTY Global, Inc.",1.0,0.0,0.0
1,Sep 15 2022 9:43AM,246496,19,"NAPP Technologies, LLC",1.0,0.0,0.0
2,Sep 15 2022 9:36AM,246494,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
3,Sep 15 2022 9:32AM,246493,12,Hush Hush,14.0,0.0,0.0
4,Sep 15 2022 9:18AM,246491,19,ACG North America LLC,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2957669,136.0,2.0,0.0
12,246493,14.0,0.0,0.0
16,739409,3.0,0.0,0.0
19,985949,6.0,0.0,0.0
20,246435,0.0,18.0,11.0
21,246291,1.0,0.0,0.0
22,246473,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2957669,136.0,2.0,0.0
1,12,246493,14.0,0.0,0.0
2,16,739409,3.0,0.0,0.0
3,19,985949,6.0,0.0,0.0
4,20,246435,0.0,18.0,11.0
5,21,246291,1.0,0.0,0.0
6,22,246473,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,136.0,2.0,0.0
1,12,14.0,0.0,0.0
2,16,3.0,0.0,0.0
3,19,6.0,0.0,0.0
4,20,0.0,18.0,11.0
5,21,1.0,0.0,0.0
6,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,136.0
1,12,Released,14.0
2,16,Released,3.0
3,19,Released,6.0
4,20,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21,22
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,11.0,0.0,0.0
Executing,2.0,0.0,0.0,0.0,18.0,0.0,1.0
Released,136.0,14.0,3.0,6.0,0.0,1.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,11.0,0.0,0.0
1,Executing,2.0,0.0,0.0,0.0,18.0,0.0,1.0
2,Released,136.0,14.0,3.0,6.0,0.0,1.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,11.0,0.0,0.0
1,Executing,2.0,0.0,0.0,0.0,18.0,0.0,1.0
2,Released,136.0,14.0,3.0,6.0,0.0,1.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()